In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy
import xgboost as xgb
import statistics as st
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import random

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv("train.csv")

In [3]:
valid_df = pd.read_csv("validation.csv")

In [ ]:
test_df = pd.read_csv("test.csv")

In [82]:
def test_results(threshold, y_p, real_clicks, payprice):
    
    pred_clicks = y_p > threshold # predicted clicks with threshold
    impressions = sum(pred_clicks)
    clicked = sum((pred_clicks + real_clicks) == 2)
    total_cost = sum(valid_df.payprice[clicks_0 == 1])
    
    ctr = clicked/impressions
    cpc = total_cost/clicked
    
    
    print("CTR is ", ctr, " impressions ", impressions, " clicks ", clicked, " total cost ", total_cost, " CPC ", cpc)
    
    #errs = len(pred_clicks) - sum(pred_clicks == real_clicks)
    #num_clicks = len(valid_df[real_clicks == 1]) + sum(pred_clicks)
    #ratio = 1-errs/num_clicks
   # clicked = (sum(pred_clicks) - sum((pred_clicks - valid_df.click) == 1))
    
   #  print("Number of predicted clicks ", sum(clicks), " with threshold set to ", threshold)
  #  print("Number of predictions matching original ", sum(clicks == real_clicks), " -> ", errs, " errors")
  #  print("Total num of clicks ", num_clicks, " -> ratio ", ratio)
  #  print("CTR ", clicked/sum(clicks), " and clicks ", clicked)
    
    return pred_clicks

In [5]:
def test_results_budget(clicks, payprice, real_clicks):
    
    sum_ones = 0
    spent = 0
    click_total = 0

    for i in range(0,len(clicks),3):
        if (clicks[i] == 1) & (spent < 6250):
            click_total = click_total + 1
            spent = spent + payprice[i]
            if real_clicks[i] == 1:
                sum_ones = sum_ones + 1

        if spent > 6250:
            break

    cpc = spent/sum_ones
    ctr_1 = sum_ones/click_total
    print("Sum payed for the clicked ones ", spent, " successful clicks ", sum_ones, " out of ", click_total)
    print("Cost per Click ", cpc)
    print("CTR ", ctr_1)

## clean data

In slotformat map Na to 10 and in adexchange map null tu 0

In [6]:
for index, c in train_df.iterrows():  
    if c.slotformat in "Na":
        train_df.set_value(index, 'slotformat', 10)
        
d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
train_df['adexchange']=train_df['adexchange'].map(d)

In [7]:
for index, c in valid_df.iterrows():  
    if c.slotformat in "Na":
        valid_df.set_value(index, 'slotformat', 10)

d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
valid_df['adexchange']=valid_df['adexchange'].map(d)

In [ ]:
for index, c in test_df.iterrows():  
    if c.slotformat in "Na":
        test_df.set_value(index, 'slotformat', 10)

d = {'1': 1, '2': 2, '3': 3 ,'4': 4, 'null': 0}
test_df['adexchange']=test_df['adexchange'].map(d)

## Constant bidding

In [8]:
const = train_df.bidprice.get_values() #[300, 294, 238, 227, 241, 277, 254, 249]
const = np.unique(const)
const = [c+1 for c in const]
const = np.sort(const)
print(const)

[228 239 242 250 255 278 295 301]


doing for training set (ctr) and validation set (ctr_v)

In [9]:
ctr = np.arange(0, len(const), dtype=np.float)
cpc = np.arange(0, len(const), dtype=np.float)
ctr_v = np.arange(0, len(const), dtype=np.float)
cpc_v = np.arange(0, len(const), dtype=np.float)
total = np.arange(0, len(const), dtype=np.float)
total_v = np.arange(0, len(const), dtype=np.float)
clicks_c = np.arange(0, len(const), dtype=np.float)
clicks_c_v = np.arange(0, len(const), dtype=np.float)
i = 0

for c in const:
    #filt is clicks/non-clicks for impressions
    filt = train_df.click[train_df.bidprice < c]
    filt_v = valid_df.click[valid_df.bidprice < c]
    
    # clickes imp / total imp
    ctr[i] = sum(filt)*100/len(filt)
    ctr_v[i] = sum(filt_v)*100/len(filt_v)
    
    # clicked imp
    clicks_c[i] = sum(filt)
    clicks_c_v[i] = sum(filt_v)  
    
    # impressions
    total[i] = len(filt)
    total_v[i] = len(filt_v)
    
    # sum payprice of imp / num of clicks
    cpc[i] = sum(train_df.payprice[train_df.bidprice < c])/clicks_c[i]
    cpc_v[i] = sum(valid_df.payprice[valid_df.bidprice < c])/clicks_c_v[i]


#    print("Price ", c)
#    print("num of clicks: ", sum(filt), "impressions: ", len(filt))
#    print("num of clicks_v: ", sum(filt_v), "impressions_v: ", len(filt_v), "\n")
    i = i + 1
    
max_index = np.argmax(ctr)
max_index_v = np.argmax(ctr_v)
print("The highest CTR for constant bidding is for ", const[max_index], " with CTR = ", ctr[max_index], \
      "% and CPC = ", cpc[max_index])

print("Numer of impressions for training set", total[max_index], " with ", clicks_c[max_index] ,\
      " clicks and total cost", (cpc[max_index])*clicks_c[max_index],"\n")

print("The highest CTR for constant bidding  in valid. set is for ", const[max_index_v], " with CTR = ", \
      ctr_v[max_index_v], "% and CPC = ", cpc_v[max_index_v])

print("Numer of impressions for validation set", total_v[max_index_v], " with ", clicks_c_v[max_index_v] ,\
      " clicks and total cost", (cpc_v[max_index_v])*clicks_c_v[max_index_v])

The highest CTR for constant bidding is for  228  with CTR =  0.0981850004625 % and CPC =  108275.126812
Numer of impressions for training set 281102.0  with  276.0  clicks and total cost 29883935.0 

The highest CTR for constant bidding  in valid. set is for  228  with CTR =  0.1112276353 % and CPC =  95139.7714286
Numer of impressions for validation set 31467.0  with  35.0  clicks and total cost 3329892.0


In [10]:
clicks_const = valid_df.bidprice < const[max_index_v]
test_results_budget(clicks_const, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6320  successful clicks  0  out of  56
Cost per Click  inf
CTR  0.0


## Random bidding

In [11]:
# 74 is the difference
train_rb = train_df
valid_rb = valid_df
upper = range(10, 80, 10) # rounded up range obtainded from max_bidprice - min_bidprice

In [24]:
random_bid_basics = np.random.rand(len(train_rb), len(upper))
name = ["bid1", "bid2", "bid3", "bid4", "bid5", "bid6", "bid7"]
ctr_r = np.arange(0, len(upper), dtype=np.float)
cpc_r = np.arange(0, len(upper), dtype=np.float)
bottom = 227
i = 0

for u in upper:
    train_rb[name[i]] = random_bid_basics[:,i]*u + bottom
   
    
    filt = train_rb.click[train_rb.bidprice < train_rb[name[i]]]
    ctr_r[i] = sum(filt)*100/len(filt)
    cpc_r[i] = sum(train_rb.payprice[train_rb.bidprice < train_rb[name[i]]])/sum(filt)
    print("num of clicks: ", sum(filt), "total: ", len(filt), " cpc ", cpc_r[i], " ctr ", ctr_r[i])
    
    i = i+1

num of clicks:  276 total:  281102  cpc  108275.126812  ctr  0.0981850004625
num of clicks:  406 total:  506771  cpc  117332.992611  ctr  0.0801150815654
num of clicks:  499 total:  667629  cpc  119760.965932  ctr  0.0747421097646
num of clicks:  549 total:  777263  cpc  123826.865209  ctr  0.0706324628858
num of clicks:  598 total:  843355  cpc  121960.242475  ctr  0.0709072691808
num of clicks:  667 total:  920076  cpc  118142.131934  ctr  0.0724940113643
num of clicks:  743 total:  990545  cpc  113724.271871  ctr  0.0750092121004


In [27]:
max_index_r = np.argmax(ctr_r)
print("The highest CTR for random bidding is for upper bound at ", upper[max_index_r], \
      " with CTR = ", ctr_r[max_index_r], "% with CPC = ", cpc_r[max_index_r])
max_index_cpc_r = np.argmax(cpc_r)
print("The highest CPC for random bidding is for upper bound at ", upper[max_index_cpc_r], \
      " with CPC = ", cpc_r[max_index_cpc_r]," and CTR = ", ctr_r[max_index_cpc_r], "%")


The highest CTR for random bidding is for upper bound at  10  with CTR =  0.0981850004625 % with CPC =  108275.126812
The highest CPC for random bidding is for upper bound at  40  with CPC =  123826.865209  and CTR =  0.0706324628858 %


In [30]:
random_bid_basics_v = np.random.rand(len(valid_rb), 1)
valid_rb["rnd_bid"] = random_bid_basics_v * upper[max_index] + bottom

auction = valid_rb.bidprice < valid_rb["rnd_bid"]
clicks_rand = valid_rb.click[valid_rb.bidprice < valid_rb["rnd_bid"]]

ctr_r_v = sum(clicks_rand)*100/len(clicks_rand)
cpc_r_v = sum(valid_rb.payprice[valid_rb.bidprice < valid_rb["rnd_bid"]])/sum(clicks_rand)

print("The CTR for random bidding on validation set is for upper bound at ", upper[max_index], \
      " is ", ctr_r_v ,"%")

print("The CPC for random bidding on validation set is for upper bound at ", upper[max_index_r], \
      " is ", cpc_r_v)

print("Total cost for valid set is ", sum(valid_rb.payprice[valid_rb.bidprice < valid_rb["rnd_bid"]]), \
      " with ", sum(auction), " impressions and ", sum(clicks_rand), " clicks \n" )

test_results_budget(auction, valid_rb.payprice, valid_rb.click)

The CTR for random bidding on validation set is for upper bound at  10  is  0.1112276353 %
The CPC for random bidding on validation set is for upper bound at  10  is  95139.7714286
Total cost for valid set is  3329892  with  31467  impressions and  35  clicks 

Sum payed for the clicked ones  6320  successful clicks  0  out of  56
Cost per Click  inf
CTR  0.0


## Linear model

In [31]:
#Define features
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y

train_df['area'] = train_df['slotwidth']*train_df['slotheight']


X_train = train_df[features]
y_train = train_df['click']

In [32]:
# Train the model to estimate pCTR
clf = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5, objective= 'binary:logistic')
clf.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [33]:
# preprocess validation set
valid_df['area'] = valid_df['slotwidth']*valid_df['slotheight']

X_valid = valid_df[features]

In [34]:
## predicts probabilities of being clicked for validation
y_p_valid = clf.predict(X_valid)

avgCTR is the average of probability that the ad will be clicked

In [35]:
avgCTR = sum(y_p_valid)/len(y_p_valid)
print("Average CTR ", avgCTR)

Average CTR  0.000755362039884


In [36]:
def calculate_bid(base_bid, y_pred, avgCTR):
    bid = np.multiply(y_pred, base_bid/avgCTR)
    return bid

In [37]:
def max_bid_base(prices, y_p_valid, avgCTR):
    ration = np.zeros(len(prices))
    total = len(valid_df)

    for i in range(len(prices)):
        tmp = calculate_bid(prices[i], y_p_valid, avgCTR) # calculates our bid price
        tmp_2 = valid_df.click[valid_df.bidprice < tmp] # stores 0/1 for those auctions we won
        if len(tmp_2) == 0:
            ration[i] = 0
        else:
            ration[i] = sum(tmp_2)/len(tmp_2) # clicked/won actions

    print("The bid_base with best CTR: ", prices[ration.argmax()])
    print("CTR for winning bid_base is ", ration.max())
    return prices[ration.argmax()]

In [38]:
prices = range(1, 301, 1)

bid_base = max_bid_base(prices, y_p_valid, avgCTR)

predicted_bid_v = calculate_bid(bid_base, y_p_valid, avgCTR)
impressions_c = valid_df.click[valid_df.bidprice < predicted_bid_v]
total_cost_c = sum(valid_df.payprice[valid_df.bidprice < predicted_bid_v])

print("Cost for valid set ", total_cost_c, ", CPC ", (total_cost_c/sum(impressions_c)), " impressions ",\
      len(impressions_c)," clicks ", sum(impressions_c))

The bid_base with best CTR:  20
CTR for winning bid_base is  0.0128479657388
Cost for valid set  40386 , CPC  6731.0  impressions  467  clicks  6


In [39]:
our_bids = calculate_bid(bid_base, y_p_valid, avgCTR) # calculates our bid price
price_payed = 0
clicked_won = 0 
won = 0

for i in range(len(our_bids)): 
    if (valid_df.bidprice[i] <= our_bids[i]):
        price_payed = price_payed + valid_df.payprice[i]
        if (price_payed < 6250):       
            won = won + 1
            if (valid_df.click[i] == 1):
                clicked_won = clicked_won + 1
        else:
            break
        
print("With budget 6250 we won ", won, " auctions and on ", clicked_won, "of them was clicked" )
print("CPC is then ", (price_payed/clicked_won))

With budget 6250 we won  66  auctions and on  1 of them was clicked
CPC is then  6281.0


## Best performing model

### quadratic

In [69]:
def calculate_bid_q(base_bid, y_pred, avgCTR):
    bid = np.multiply(y_pred**2, (base_bid*150)/avgCTR)
    return bid

In [57]:
def max_bid_base_q(prices, y_p_valid, avgCTR):
    ration = np.zeros(len(prices))
    total = len(valid_df)

    for i in range(len(prices)):
        tmp = calculate_bid_q(prices[i], y_p_valid, avgCTR) # calculates our bid price
        tmp_2 = valid_df.click[valid_df.bidprice < tmp] # stores 0/1 for those auctions we won

        if len(tmp_2) == 0:
            ration[i] = 0
        else:
            ration[i] = sum(tmp_2)/len(tmp_2) # clicked/won actions

    print("The bid_base with best CTR: ", prices[ration.argmax()])
    print("CTR for winning bid_base is ", ration.max())
    
    return prices[ration.argmax()]

In [70]:
prices_q = range(1, 450, 1)

bid_base_q = max_bid_base(prices_q, y_p_valid, avgCTR)

predicted_bid_v_q = calculate_bid_q(bid_base_q, y_p_valid, avgCTR)
impressions_q = valid_df.click[valid_df.bidprice < predicted_bid_v_q]
total_cost_q = sum(valid_df.payprice[valid_df.bidprice < predicted_bid_v_q])

print("Cost for valid set ", total_cost_q, ", CPC ", (total_cost_q/sum(impressions_q)), " impressions ",\
      len(impressions_q)," clicks ", sum(impressions_q))

The bid_base with best CTR:  13
CTR for winning bid_base is  0.0125260960334
Cost for valid set  41115 , CPC  6852.5  impressions  479  clicks  6


### prediction with hour and weekday separated

In [71]:
# Train the model to estimate pCTR
clf_0 = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_0.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [72]:
## predicts probabilities of being clicked for validation
y_p_valid_0 = clf_0.predict(X_valid)

In [83]:
threshold_0 = 0.01

clicks_0 = test_results(threshold_0, y_p_valid_0, valid_df["click"], valid_df["payprice"])


CTR is  0.0097244732577  impressions  617  clicks  6  total cost  54635  CPC  9105.83333333


In [81]:
test_results_budget(clicks_0, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6269  successful clicks  1  out of  73
Cost per Click  6269.0
CTR  0.0136986301369863


### prediction for weekday, hour and weekday+hour

In [84]:
# encoding weekday and hour into one variable
X_train['weekday']=X_train['weekday'].astype(str)
X_train['hour']=X_train['hour'].astype(str)

X_train['time'] = X_train[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_train = X_train.astype("int")

#X_train = X_train.drop(["hour", "weekday"], axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [85]:
# encoding weekday and hour into one variable
X_valid['weekday']=X_valid['weekday'].astype(str)
X_valid['hour']=X_valid['hour'].astype(str)

X_valid['time'] = X_valid[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_valid = X_valid.astype("int")

#X_valid = X_valid.drop(["hour", "weekday"], axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [86]:
# Train the model to estimate pCTR
clf_1 = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_1.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [87]:
## predicts probabilities of being clicked for validation
y_p_valid_1 = clf_1.predict(X_valid)

In [88]:
threshold_1 = 0.01

clicks_1 = test_results(threshold_1, y_p_valid_1, valid_df["click"], valid_df["payprice"])

CTR is  0.0117370892019  impressions  426  clicks  5  total cost  54635  CPC  10927.0


In [89]:
test_results_budget(clicks_1, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6412  successful clicks  3  out of  67
Cost per Click  2137.33333333
CTR  0.04477611940298507


## XGBoost chosen atributes

In [90]:
#Define features
features_a = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser"]
# Prepare X and y

train_df['area'] = train_df['slotwidth']*train_df['slotheight']

X_train_a = train_df[features_a]              
              
X_train_a['weekday']=X_train_a['weekday'].astype(str)
X_train_a['hour']=X_train_a['hour'].astype(str)

X_train_a['time'] = X_train_a[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_train_a = X_train_a.astype("int")
X_train_a = X_train_a.drop(["hour", "weekday"], axis=1)

y_train_a = train_df['click']

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versu

In [91]:
clf_a = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_a.fit(X_train_a, y_train_a)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [92]:
valid_df['area'] = valid_df['slotwidth']*valid_df['slotheight']

X_valid_a = valid_df[features_a]              
              
X_valid_a['weekday']=X_valid_a['weekday'].astype(str)
X_valid_a['hour']=X_valid_a['hour'].astype(str)

X_valid_a['time'] = X_valid_a[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_valid_a = X_valid_a.astype("int")
X_valid_a = X_valid_a.drop(["hour", "weekday"], axis=1)


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [93]:
y_p_valid_a = clf_a.predict(X_valid_a)

In [95]:
clicks = test_results(0.01, y_p_valid_a, valid_df.click, valid_df.payprice)

CTR is  0.00378787878788  impressions  528  clicks  2  total cost  54635  CPC  27317.5


In [96]:
test_results_budget(clicks, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6311  successful clicks  1  out of  70
Cost per Click  6311.0
CTR  0.014285714285714285


## Undersampling

In [97]:
ratio_1_0 = sum(train_df.click)/len(train_df)
print("Percentage of 1s in dataset train ", ratio_1_0*100)
subset = train_df.sample(frac=0.001)
subset = pd.concat([subset, train_df[train_df.click == 1]])

Percentage of 1s in dataset train  0.0753964988446


In [98]:
#Define features
features = ["weekday", "hour", "region", "city", "area", "slotprice", "advertiser", "adexchange"]
# Prepare X and y


X_train_us = subset[features]
y_train_us = subset['click']

# encoding weekday and hour into one variable
X_train_us['weekday']=X_train_us['weekday'].astype(str)
X_train_us['hour']=X_train_us['hour'].astype(str)

X_train_us['time'] = X_train_us[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_train_us = X_train_us.astype("int")

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [99]:
clf_us = xgb.XGBRegressor(n_estimators=100, max_depth = 3, learning_rate=0.5)
clf_us.fit(X_train_us, y_train_us)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [100]:
X_valid_us = valid_df[features]

# encoding weekday and hour into one variable
X_valid_us['weekday']=X_valid_us['weekday'].astype(str)
X_valid_us['hour']=X_valid_us['hour'].astype(str)

X_valid_us['time'] = X_valid_us[['weekday', 'hour']].apply(lambda x: ''.join(x), axis=1)

X_valid_us = X_valid_us.astype("int")

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [101]:
y_predict_us = clf_1.predict(X_valid_us)

In [102]:
threshold_us = 0.01

clicks_us = test_results(threshold_us, y_predict_us, valid_df["click"], valid_df["payprice"])

CTR is  0.0117370892019  impressions  426  clicks  5  total cost  54635  CPC  10927.0


In [103]:
test_results_budget(clicks_us, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6412  successful clicks  3  out of  67
Cost per Click  2137.33333333
CTR  0.04477611940298507


## Undersampled RF

In [104]:
# Set the random seed
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

random_forest = RandomForestClassifier(n_estimators=1000, max_features=2, oob_score=True)

In [105]:
random_forest.fit(X=X_train_us, y=y_train_us)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [106]:
y_rf = random_forest.predict(X_valid_us)

In [107]:
clicks_rf = test_results(0.5, y_rf, valid_df.click, valid_df.payprice)

CTR is  0.00132256265965  impressions  77123  clicks  102  total cost  54635  CPC  535.637254902


In [108]:
test_results_budget(clicks_rf, valid_df.payprice, valid_df.click)

Sum payed for the clicked ones  6344  successful clicks  0  out of  67
Cost per Click  inf
CTR  0.0
